# NLP Rules Notebook: A Brief Introduction


In this notebook, we'll implement the final component of our F1 Strategy Expert System: rules that analyze team radio communications using Natural Language Processing (NLP).

These rules will complement our existing tire degradation rules (N02) and lap time rules (N03) by incorporating qualitative information from team communications.

---

## 1. Setting Up the Environment

In [1]:
# Import standard Python libraries
import pandas as pd              # For data manipulation and analysis
import numpy as np               # For numerical operations
import matplotlib.pyplot as plt  # For creating visualizations
import seaborn as sns            # For enhanced visualizations
import json                      # For handling JSON data from NLP analysis
from datetime import datetime    # For timestamp handling
import os                        # For operating system interactions
import sys                       # For system-specific parameters and functions

# Add parent directory to system path to make custom modules accessible
sys.path.append(os.path.abspath('../'))  

# Import Experta components for building the rule engine
from experta import Rule, NOT, OR, AND, AS, MATCH, TEST, EXISTS  # Rule definition components
from experta import DefFacts, Fact, Field, KnowledgeEngine      # Core Experta classes

# Import custom fact classes and functions from previous notebooks
import utils.N01_agent_setup as agent_setup
from utils.N01_agent_setup import (
    TelemetryFact,              # For storing car performance data
    DegradationFact,            # For storing tire degradation information
    RaceStatusFact,             # For storing current race conditions
    RadioFact,                  # For storing NLP analysis of radio messages
    StrategyRecommendation,     # For storing strategy recommendations
    F1StrategyEngine,           # Base engine class
    transform_radio_analysis,   # Function to convert NLP analysis to facts
    process_radio_message       # Function to process raw radio messages
)

# Try importing the NLP radio processing module if available

from NLP_radio_processing.NLP_utils import N06_model_merging as radio_nlp
  

# Configure plotting style for better visualization
plt.style.use('seaborn-v0_8-darkgrid')  # Set default plot style
sns.set_context("notebook", font_scale=1.2)  # Increase font size slightly

print("Libraries and fact classes loaded successfully.")

Libraries and fact classes loaded successfully.


---

## 2.  the radio analysis pipeline 

This function is already defined in `N01_agent_setup.ipynb`

### Function review:

1. `process_radio_message(message, is_audio = False)`:
    - Takes a text message or audio file path.
    - Returns path to JSON file with analysis.

2. `transform_radio_analysis(radio_json_path)`:
    - Takes path to JSON file with analysis.
    - Returns RadioFact object.

3. `analyze_and_transform_radio(message, is_audio = False)`:
    - Combined function: message -> RadioFact

### 2.1 Processing a Radio Sample 

In [2]:
sample_message = "Box this lap for softs, there's rain expected in 10 minutes"
print(f"Processing sample message: '{sample_message}'")
# Use the function from N01_agent_setup to process the message
json_path = process_radio_message(sample_message)

Processing sample message: 'Box this lap for softs, there's rain expected in 10 minutes'
Analyzing message: 'Box this lap for softs, there's rain expected in 10 minutes'

Analyzing message: "Box this lap for softs, there's rain expected in 10 minutes"

Detected entities:
  PIT_CALL:
    • "Box this lap for softs,"
  WEATHER:
    • "there's rain expected in 10 minutes"
Analysis saved to ../../outputs/week4/json/radio_analysis_20250427_181454.json


### 2.2 Transforming the radio analysis.


In [3]:
if json_path:
    print(f"Processing json from: {json_path}")
    
    # Use the function from N01_agent_setup to transform JSON to RadioFact
    radio_fact = transform_radio_analysis(json_path)
    
    # Load the JSON file again to display its structure
    with open(json_path, 'r') as f:
        radio_analysis = json.load(f)
else:
    print("Failed to process message. Using sample data instead.")
    radio_fact = None
    radio_analysis = None

Processing json from: ../../outputs/week4/json/radio_analysis_20250427_181454.json


### 2.3 Examine the structure of the radio analysis

In [4]:
print("\nExamining radio analysis structure:")
print(f"Message: '{radio_analysis['message']}'")
print(f"Sentiment: {radio_analysis['analysis']['sentiment']}")
print(f"Intent: {radio_analysis['analysis']['intent']}")


Examining radio analysis structure:
Message: 'Box this lap for softs, there's rain expected in 10 minutes'
Sentiment: neutral
Intent: INFORMATION


### 2.4 Entity categories and their values

In [5]:
# Display the entity categories and their values
print("\nEntity categories in analysis:")
for category, entities in radio_analysis["analysis"]["entities"].items():
    if entities:  # Only show non-empty categories
        print(f"  - {category}: {entities}")


Entity categories in analysis:
  - PIT_CALL: ['Box this lap for softs,']
  - WEATHER: ["there's rain expected in 10 minutes"]


### 2.5 Examining the RadioFact that was created

In [6]:
# Examine the RadioFact that was created
print("\nRadioFact created from analysis:")
print(f"Sentiment: {radio_fact['sentiment']}")
print(f"Intent: {radio_fact['intent']}")
print(f"Entity categories: {list(radio_fact['entities'].keys())}")


RadioFact created from analysis:
Sentiment: neutral
Intent: INFORMATION
Entity categories: ['PIT_CALL', 'WEATHER']


---

## 3. Implementing Radio-Based Rules 

In this section, I´ll implement the F1RadioRules class that inherits from F1StrategyEngine. The rules to analyze radio coommunications are going to be these three ones.

1. *Weather Response Rules*.
2. *Grip/Tire Issues Rules*.
3. *Track Incident Rules*.

Each rule will follow the Experta pattern with conditions based on the RadioFact data (sentiment, intent, entities) and produce appropiate strategy recommendations.

In [7]:
class F1RadioRules(F1StrategyEngine):
    """
    Final version of rules for F1 Strategy based on NLP analysis of radio communications.
    
    Rules have been adjusted based on actual output patterns from the NLP models and
    improved to provide clearer explanations.
    """
    
    ##################################### RULE 1: Response to Grip Issues (Updated)
    @Rule(
        # Match Radio Facts with negative sentiment
        RadioFact(sentiment="negative"),
        
        # Capture the entities dictionary
        RadioFact(entities=MATCH.entities),
        
        # Updated test to look for grip-related terms in ANY entity category
        # or check for TECHNICAL_ISSUE entities which indicate car problems
        TEST(lambda entities: 
             any('grip' in str(value).lower() for category, values in entities.items() for value in values) or
             any('struggle' in str(value).lower() for category, values in entities.items() for value in values) or
             len(entities.get('TECHNICAL_ISSUE', [])) > 0),
        
        # Capture current lap for the recommendation
        RaceStatusFact(lap=MATCH.lap)
    )
    def grip_issue_response(self, entities, lap):
        """
        Final Rule 1: Response to Grip Issues
        
        Looks for grip-related terms in any entity category or checks for TECHNICAL_ISSUE entities
        which typically indicate car problems. Provides a qualitative recommendation based solely
        on driver feedback.
        """
        # Debug output to show rule activation and conditions
        print(f"Rule activated: grip_issue_response")
        print(f"  - Sentiment: negative")
        
        # Find all grip-related entities or technical issues
        grip_related = []
        for category, values in entities.items():
            for value in values:
                if 'grip' in str(value).lower() or 'struggle' in str(value).lower():
                    grip_related.append(f"{value} [{category}]")
        
        # Add any technical issues
        tech_issues = entities.get('TECHNICAL_ISSUE', [])
        for issue in tech_issues:
            if issue not in grip_related:
                grip_related.append(f"{issue} [TECHNICAL_ISSUE]")
        
        print(f"  - Grip-related entities: {grip_related}")
        
        # Get the most relevant issue for the explanation
        issue_text = "grip issues"
        if grip_related:
            issue_text = grip_related[0].split(" [")[0]  # Remove the category tag
        
        # Create and declare the recommendation with improved explanation
        self.declare(
            StrategyRecommendation(
                action="prioritize_pit",
                confidence=0.85,  # 85% confidence as specified
                explanation=f"Driver reports degradation issues. Recommend confirming with telemetry data and increasing pit stop priority if degradation is confirmed.",
                priority=2,  # Medium-high priority - needs data confirmation
                lap_issued=lap
            )
        )
        
        # Record that this rule was activated
        self.record_rule_fired("grip_issue_response")
    
    ##################################### RULE 2: Weather Information Adjustment (Unchanged)
    @Rule(
        # Capture the entities dictionary - no longer requiring WARNING intent
        RadioFact(entities=MATCH.entities),
        
        # Test for rain/wet in WEATHER or TRACK_CONDITION entities
        TEST(lambda entities: 
             len(entities.get('WEATHER', [])) > 0 or
             any('wet' in condition.lower() for condition in entities.get('TRACK_CONDITION', []))),
        
        # Capture current lap for the recommendation
        RaceStatusFact(lap=MATCH.lap)
    )
    def weather_information_adjustment(self, entities, lap):
        """
        Final Rule 2: Weather Information Adjustment
        
        Focuses directly on WEATHER entities or wet TRACK_CONDITION entities
        rather than relying on the intent classification.
        """
        # Debug output to show rule activation and conditions
        print(f"Rule activated: weather_information_adjustment")
        
        # Get weather entities
        weather_entities = entities.get('WEATHER', [])
        print(f"  - WEATHER entities: {weather_entities}")
        
        # Get track condition entities that mention wetness
        wet_conditions = [condition for condition in entities.get('TRACK_CONDITION', []) 
                         if 'wet' in condition.lower()]
        print(f"  - Wet TRACK_CONDITION entities: {wet_conditions}")
        
        # Combine the weather mentions for the explanation
        weather_text = []
        weather_text.extend(weather_entities)
        weather_text.extend(wet_conditions)
        
        if not weather_text:
            weather_text = ["changing weather conditions"]
            
        # Create and declare the recommendation
        self.declare(
            StrategyRecommendation(
                action="prepare_rain_tires",
                confidence=0.9,  # 90% confidence as specified
                explanation=f"Weather change detected: {', '.join(weather_text)}. Prepare strategy for switch to rain tires.",
                priority=3,  # High priority as weather changes require immediate attention
                lap_issued=lap
            )
        )
        
        # Record that this rule was activated
        self.record_rule_fired("weather_information_adjustment")
    
    ##################################### RULE 3: Incident Reaction (Unchanged)
    @Rule(
        # Capture the entities dictionary
        RadioFact(entities=MATCH.entities),
        
        # Test for 'safety' in INCIDENT or 'yellow' in SITUATION entities
        TEST(lambda entities: 
             any('safety' in incident.lower() for incident in entities.get('INCIDENT', [])) or
             any('yellow' in situation.lower() for situation in entities.get('SITUATION', []))),
        
        # Capture current lap for the recommendation
        RaceStatusFact(lap=MATCH.lap)
    )
    def incident_reaction(self, entities, lap):
        """
        Final Rule 3: Incident Reaction
        
        Identifies safety car or yellow flag situations and recommends strategic pit window evaluation.
        """
        # Debug output to show rule activation and conditions
        print(f"Rule activated: incident_reaction")
        
        # Get safety car mentions
        safety_incidents = [incident for incident in entities.get('INCIDENT', []) 
                           if 'safety' in incident.lower()]
        
        # Get yellow flag mentions
        yellow_situations = [situation for situation in entities.get('SITUATION', []) 
                            if 'yellow' in situation.lower()]
        
        print(f"  - INCIDENT entities with 'safety': {safety_incidents}")
        print(f"  - SITUATION entities with 'yellow': {yellow_situations}")
        
        # Determine the type of incident for the explanation
        if safety_incidents:
            incident_type = safety_incidents[0]
        elif yellow_situations:
            incident_type = yellow_situations[0]
        else:
            incident_type = "race neutralization"
            
        # Create and declare the recommendation
        self.declare(
            StrategyRecommendation(
                action="reevaluate_pit_window",
                confidence=0.85,  # 85% confidence as specified
                explanation=f"Race incident detected: {incident_type}. Strategic opportunity to pit during race neutralization.",
                priority=3,  # High priority as these are time-sensitive opportunities
                lap_issued=lap
            )
        )
        
        # Record that this rule was activated
        self.record_rule_fired("incident_reaction")

---

## 4. Testing the Radio Rules.

In [8]:
def test_radio_rules(scenario_name, message, additional_facts=None):
    """
    Test the radio rules with a specific message and scenario
    
    Args:
        scenario_name: Name of the test scenario
        message: Radio message to analyze
        additional_facts: Dictionary of additional facts to declare (optional)
        
    Returns:
        List of recommendations generated by the rules
    """
    print(f"\n{'='*80}")
    print(f"TESTING SCENARIO: {scenario_name}")
    print(f"{'='*80}")
    print(f"Radio message: '{message}'")
    
    # Step 1: Process the message using our NLP pipeline
    print("\nProcessing message with NLP pipeline...")
    json_path = radio_nlp.analyze_radio_message(message)
    
    if not json_path:
        print("Failed to process message.")
        return []
        
    print(f"Analysis saved to: {json_path}")
    
    # Load the analysis
    with open(json_path, 'r') as f:
        radio_analysis = json.load(f)
        
    # Extract key components for debugging
    sentiment = radio_analysis["analysis"]["sentiment"]
    intent = radio_analysis["analysis"]["intent"]
    
    # Show extracted entities
    entities = radio_analysis["analysis"]["entities"]
    non_empty_entities = {k: v for k, v in entities.items() if v}
    
    print(f"\nAnalysis results:")
    print(f"Sentiment: {sentiment}")
    print(f"Intent: {intent}")
    print("Entities detected:")
    for category, values in non_empty_entities.items():
        print(f"  - {category}: {values}")
            
    # Create RadioFact
    radio_fact = transform_radio_analysis(json_path)
    
    # Step 2: Initialize the rule engine
    print("\nInitializing rule engine...")
    engine = F1RadioRules()
    engine.reset()
    
    # Step 3: Declare the RadioFact
    engine.declare(radio_fact)
    
    # Step 4: Declare additional facts if provided
    if additional_facts:
        print("Declaring additional facts:")
        for fact_name, fact in additional_facts.items():
            print(f"  - {fact_name}: {fact}")
            engine.declare(fact)
    
    # Step 5: Run the engine
    print("\nRunning rule engine...")
    engine.run()
    
    # Step 6: Get and display recommendations
    recommendations = engine.get_recommendations()
    
    print(f"\nGenerated {len(recommendations)} recommendations:")
    if recommendations:
        for i, rec in enumerate(recommendations):
            print(f"\nRecommendation {i+1}:")
            print(f"Action: {rec['action']}")
            print(f"Confidence: {rec['confidence']}")
            print(f"Explanation: {rec['explanation']}")
            print(f"Priority: {rec['priority']}")
            print(f"Lap issued: {rec['lap_issued']}")
    else:
        print("No recommendations generated.")
    
    # Show which rules were triggered
    print("\nTriggered rules:")
    if engine.rules_fired:
        for rule in engine.rules_fired:
            print(f"- {rule['rule']} (Lap {rule['lap']})")
    else:
        print("No rules were triggered.")
    
    return recommendations



In [9]:
# Now let's test each rule with a specific scenario

# Test 1: Grip issue rule
print("\nPreparing Test 1: Grip Issues Rule")
grip_message = "I'm really struggling with grip, the rear feels terrible"
# We need to add a DegradationFact with rate > 0.09 to trigger the rule
grip_additional_facts = {
    "race_status": RaceStatusFact(lap=20, total_laps=60)
}
grip_recommendations = test_radio_rules("Grip Issues", grip_message, grip_additional_facts)




Preparing Test 1: Grip Issues Rule

TESTING SCENARIO: Grip Issues
Radio message: 'I'm really struggling with grip, the rear feels terrible'

Processing message with NLP pipeline...

Analyzing message: "I'm really struggling with grip, the rear feels terrible"

Detected entities:
  INCIDENT:
    • "struggling with"
  SITUATION:
    • "I'm"
  TECHNICAL_ISSUE:
    • "the"
    • "rear feels terrible"
Analysis saved to ../../outputs/week4/json/radio_analysis_20250427_181501.json
Analysis saved to: ../../outputs/week4/json/radio_analysis_20250427_181501.json

Analysis results:
Sentiment: negative
Intent: PROBLEM
Entities detected:
  - SITUATION: ["I'm"]
  - INCIDENT: ['struggling with']
  - TECHNICAL_ISSUE: ['the', 'rear feels terrible']

Initializing rule engine...
Declaring additional facts:
  - race_status: <Undeclared Fact> RaceStatusFact(lap=20, total_laps=60)

Running rule engine...
Rule activated: grip_issue_response
  - Sentiment: negative
  - Grip-related entities: ['the [TECHNICAL

In [10]:
# Test 2: Weather warning rule
print("\nPreparing Test 2: Weather Warning Rule")
weather_message = "Warning: rain starting at turn 4, track is getting wet"
weather_additional_facts = {
    "race_status": RaceStatusFact(lap=25, total_laps=60)
}
weather_recommendations = test_radio_rules("Weather Warning", weather_message, weather_additional_facts)




Preparing Test 2: Weather Warning Rule

TESTING SCENARIO: Weather Warning
Radio message: 'Warning: rain starting at turn 4, track is getting wet'

Processing message with NLP pipeline...

Analyzing message: "Warning: rain starting at turn 4, track is getting wet"

Detected entities:
  TRACK_CONDITION:
    • "track is getting wet"
  WEATHER:
    • "rain starting"
Analysis saved to ../../outputs/week4/json/radio_analysis_20250427_181507.json
Analysis saved to: ../../outputs/week4/json/radio_analysis_20250427_181507.json

Analysis results:
Sentiment: neutral
Intent: INFORMATION
Entities detected:
  - WEATHER: ['rain starting']
  - TRACK_CONDITION: ['track is getting wet']

Initializing rule engine...
Declaring additional facts:
  - race_status: <Undeclared Fact> RaceStatusFact(lap=25, total_laps=60)

Running rule engine...
Rule activated: weather_information_adjustment
  - WEATHER entities: frozenlist(['rain starting',])
  - Wet TRACK_CONDITION entities: ['track is getting wet']

Generat

In [11]:
# Test 3: Safety car/incident rule
print("\nPreparing Test 3: Safety Car Rule")
incident_message = "Safety car has been deployed, box box box"
incident_additional_facts = {
    "race_status": RaceStatusFact(lap=30, total_laps=60)
}
incident_recommendations = test_radio_rules("Safety Car", incident_message, incident_additional_facts)




Preparing Test 3: Safety Car Rule

TESTING SCENARIO: Safety Car
Radio message: 'Safety car has been deployed, box box box'

Processing message with NLP pipeline...

Analyzing message: "Safety car has been deployed, box box box"

Detected entities:
  INCIDENT:
    • "Safety"
  PIT_CALL:
    • "box box box"
Analysis saved to ../../outputs/week4/json/radio_analysis_20250427_181514.json
Analysis saved to: ../../outputs/week4/json/radio_analysis_20250427_181514.json

Analysis results:
Sentiment: negative
Intent: PROBLEM
Entities detected:
  - INCIDENT: ['Safety']
  - PIT_CALL: ['box box box']

Initializing rule engine...
Declaring additional facts:
  - race_status: <Undeclared Fact> RaceStatusFact(lap=30, total_laps=60)

Running rule engine...
Rule activated: incident_reaction
  - INCIDENT entities with 'safety': ['Safety']
  - SITUATION entities with 'yellow': []

Generated 1 recommendations:

Recommendation 1:
Action: reevaluate_pit_window
Confidence: 0.85
Explanation: Race incident dete

In [12]:
# Summarize results
print("\n" + "="*50)
print("SUMMARY OF RULE TESTING")
print("="*50)
print(f"1. Grip Issues Rule: {len(grip_recommendations)} recommendations generated")
print(f"2. Weather Warning Rule: {len(weather_recommendations)} recommendations generated")
print(f"3. Safety Car Rule: {len(incident_recommendations)} recommendations generated")
print("\nTotal Rules Tested: 3")

# Calculate success rate
triggered_rules = sum(1 for recs in [grip_recommendations, weather_recommendations, incident_recommendations] if recs)
print(f"Rules Successfully Triggered: {triggered_rules}/3 ({triggered_rules/3*100:.0f}%)")


SUMMARY OF RULE TESTING
1. Grip Issues Rule: 1 recommendations generated
2. Weather Warning Rule: 1 recommendations generated
3. Safety Car Rule: 1 recommendations generated

Total Rules Tested: 3
Rules Successfully Triggered: 3/3 (100%)


---

# Summary: NLP Radio Rules for F1 Strategy

In this notebook, we've successfully implemented a rule-based expert system that analyzes Formula 1 team radio communications to make strategic recommendations. This adds a critical qualitative dimension to our F1 strategy system.

## Key Accomplishments

1. **Radio Data Integration**
   - Connected our expert system to the NLP radio analysis pipeline.
   - Processed structured data from radio messages with sentiment, intent, and entity information.
   - Transformed this data into RadioFact objects for use in our rule engine.

2. **Strategic Rules Implementation**
   - Implemented three key radio-based rules:
     - **Grip Issue Response**: Identifies when drivers report tire problems.
     - **Weather Information Adjustment**: Detects changing weather conditions.
     - **Incident Reaction**: Recognizes safety car and yellow flag opportunities.

3. **Testing and Refinement**
   - Tested the rules with real radio messages.
   - Refined the rule conditions based on actual NLP output patterns.
   - Achieved successful activation of all three rules with our mock examples.

This notebook completes the third major component of our F1 strategy expert system, providing it with the ability to incorporate the critical information communicated over team radio, which often contains early warnings and strategic context not available in the telemetry data alone.